In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import qutip as qt
from qutip import *

# Physical Larmor frequency
f_eZ = 50e6  # 50 MHz
omega_eZ = 2 * np.pi * f_eZ  # 50 MHz

# Hamiltonian: H = (omega0/2) * sigma_z
H = 0.5 * omega_eZ * sigmaz()

# Initial state |+x>
up = basis(2,0)
down = basis(2,1)
psi0 = (up + down).unit() # 1/\sqrt(2) (|0> + |1>)
#psi0 = basis(2,0)

# Time grid: a few Larmor cycles
T = 1 / f_eZ               # period in seconds (20 ns)
tlist = np.linspace(0, 3*T, 2000)

# Observables
ops = [sigmax(), sigmay(), sigmaz()]

# Solve Schrödinger equation (unitary)
result = sesolve(H, psi0, tlist, e_ops=ops)

sx, sy, sz = result.expect

# Plot expectations
plt.plot(tlist, sx, label='sx')
plt.plot(tlist, sy, label='sy')
plt.plot(tlist, sz, label='sz')
plt.xlabel('time (s)')
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import basis, tensor, sigmax, sigmay, sigmaz, sesolve, expect

# ------------------------
# Parameters
# ------------------------
omega0 = 2*np.pi*50e6      # Electron Larmor freq (rad/s)
A = 2*np.pi*5e6            # Hyperfine coupling (rad/s)
n_nuclei = 3

# Electron spin operators
sx = sigmax()
sy = sigmay()
sz = sigmaz()

# Nuclear spin operators (each as sigma_z)
I_z_list = [sigmaz() for _ in range(n_nuclei)]

# Build total Sz for nuclei
# Assume nuclei polarized up (state |up> -> +1/2)
# Construct tensor product basis: |e> ⊗ |n1> ⊗ |n2> ⊗ |n3>
# We'll construct operators using tensor identities
def electron_op(op):
    return tensor([op] + [qeye(2) for _ in range(n_nuclei)])
def nuclear_op(k, op):
    return tensor([qeye(2) if i!=k else op for i in range(n_nuclei+1)])

# ------------------------
# Hamiltonian
# ------------------------
# Electron Zeeman
H_e = 0.5 * omega0 * electron_op(sz)

# Hyperfine: S·Iz (only z component for static polarization)
H_hf = sum( 0.5 * A * electron_op(sz) * nuclear_op(k, sz) for k in range(n_nuclei) )

H = H_e + H_hf

# ------------------------
# Initial state: electron along x, nuclei all up
# ------------------------
up = basis(2,0)
down = basis(2,1)

# Electron in +x = (|up> + |down>)/sqrt(2)
psi_e = (up + down).unit()
# Nuclei all up
psi_n = tensor([up for _ in range(n_nuclei)])
# Total initial state
psi0 = tensor(psi_e, psi_n)

# ------------------------
# Time evolution
# ------------------------
tlist = np.linspace(0, 200e-9, 500)  # 200 ns total
result = sesolve(H, psi0, tlist, e_ops=[electron_op(sx), electron_op(sy), electron_op(sz)])

sx_exp = result.expect[0]
sy_exp = result.expect[1]
sz_exp = result.expect[2]

# ------------------------
# Plot results
# ------------------------
plt.figure(figsize=(6,4))
plt.plot(tlist*1e9, sx_exp, label='<Sx>')
plt.plot(tlist*1e9, sy_exp, label='<Sy>')
plt.plot(tlist*1e9, sz_exp, label='<Sz>')
plt.xlabel('Time (ns)')
plt.ylabel('Expectation values')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import basis, tensor, sigmax, sigmay, sigmaz, sesolve, qeye, expect

# ------------------------
# Parameters
# ------------------------
omega0 = 2*np.pi*50e6      # Electron Larmor frequency (rad/s)
A = 2*np.pi*5e6            # Hyperfine coupling constant (rad/s)
tlist = np.linspace(0, 200e-9, 500)  # 200 ns

# Spin operators
sx = sigmax()
sy = sigmay()
sz = sigmaz()
up = basis(2,0)
down = basis(2,1)

# Helper functions
def electron_op(op, n_nuc):
    return tensor([op] + [qeye(2) for _ in range(n_nuc)])
def nuclear_op(k, op, n_nuc):
    return tensor([qeye(2) if i!=k else op for i in range(n_nuc+1)])

# ------------------------
# 1. Show Overhauser shift for 2 fixed configurations
# ------------------------
n_nuclei = 3

# Electron initial state along x
psi_e = (up + down).unit()

# Two nuclear configs
configs = [
    tensor([up for _ in range(n_nuclei)]),
    tensor([down, up, up])  # one flipped
]

plt.figure(figsize=(6,4))
for config in configs:
    psi0 = tensor(psi_e, config)
    # Hamiltonian
    H = 0.5*omega0*electron_op(sz, n_nuclei)
    H += sum(0.5*A*electron_op(sz,n_nuclei)*nuclear_op(k,sz,n_nuclei) for k in range(n_nuclei))
    # Time evolution
    result = sesolve(H, psi0, tlist, e_ops=[electron_op(sx, n_nuclei)])
    plt.plot(tlist*1e9, result.expect[0], label=f'Config: {[1 if expect(sz, config.ptrace(k)) > 0 else 0 for k in range(n_nuclei)]}')
plt.xlabel('Time (ns)')
plt.ylabel('<Sx>')
plt.title('Shifted Precession: Two Nuclear Configurations')
plt.legend()
plt.show()

# ------------------------
# 2. Visualize dephasing over random nuclear ensemble
# ------------------------
n_nuclei = 6
n_samples = 100
sx_avg = np.zeros_like(tlist)

for _ in range(n_samples):
    # Random nuclear config: spin up (+1/2) or down (-1/2)
    nuc_state_list = [up if np.random.rand()>0.5 else down for _ in range(n_nuclei)]
    psi_n = tensor(nuc_state_list)
    psi0 = tensor(psi_e, psi_n)
    H = 0.5*omega0*electron_op(sz,n_nuclei)
    H += sum(0.5*A*electron_op(sz,n_nuclei)*nuclear_op(k,sz,n_nuclei) for k in range(n_nuclei))
    result = sesolve(H, psi0, tlist, e_ops=[electron_op(sx,n_nuclei)])
    sx_avg += result.expect[0]

sx_avg /= n_samples

plt.figure(figsize=(6,4))
plt.plot(tlist*1e9, sx_avg)
plt.xlabel('Time (ns)')
plt.ylabel('Ensemble-averaged <Sx>')
plt.title(f'Dephasing due to quasi-static Overhauser field (N={n_nuclei})')
plt.show()

# ------------------------
# 3. Visualize linewidth (Fourier transform)
# ------------------------
fft = np.fft.fft(sx_avg)
freqs = np.fft.fftfreq(len(tlist), d=(tlist[1]-tlist[0]))
plt.figure(figsize=(6,4))
plt.plot(freqs/1e6, np.abs(fft))
plt.xlabel('Frequency (MHz)')
plt.ylabel('Amplitude')
plt.title('Fourier Transform → Linewidth')
plt.xlim(0, 200)  # zoom in
plt.show()

# ------------------------
# 4. Shift with number of nuclei
# ------------------------
n_max = 12
shifts = []

for n in range(1, n_max+1):
    # Single random nuclear configuration
    nuc_state_list = [up for _ in range(n)]
    psi_n = tensor(nuc_state_list)
    psi0 = tensor(psi_e, psi_n)
    H = 0.5*omega0*electron_op(sz,n)
    H += sum(0.5*A*electron_op(sz,n)*nuclear_op(k,sz,n) for k in range(n))
    result = sesolve(H, psi0, tlist, e_ops=[electron_op(sx,n)])
    # Estimate precession frequency from first peak
    sx_data = result.expect[0]
    dt = tlist[1]-tlist[0]
    peaks = np.where(np.diff(np.sign(np.diff(sx_data)))<0)[0]  # local maxima
    if len(peaks)>=2:
        period = (peaks[1]-peaks[0])*dt
        shifts.append(1/period - omega0/(2*np.pi))  # relative shift in Hz
    else:
        shifts.append(0)

plt.figure(figsize=(6,4))
plt.plot(range(1,n_max+1), np.array(shifts)*1e-6, 'o-')
plt.xlabel('Number of nuclei')
plt.ylabel('Overhauser shift (MHz)')
plt.title('Frequency shift vs number of nuclei')
plt.show()


In [ ]:
# Flip-Flop Simulation
import numpy as np
from qutip import *

# --- Parameters ---
A = 1 * 2*np.pi       # hyperfine strength (arbitrary but strong to see oscillation) Change to A = 10e6 * 2*np.pi (1 MHz)
omega_e = 0.0           # no Zeeman → flip-flop allowed
N = 2                   # number of nuclei

# --- Spin operators ---
sx, sy, sz = sigmax(), sigmay(), sigmaz()
sp, sm = sigmap(), sigmam()

# electron operators (tensor with identities)
def e(op):
    ops = [op] + [qeye(2) for _ in range(N)]
    return tensor(ops)

# nuclear operator k
def n(op, k):
    ops = [qeye(2)] + [qeye(2) for _ in range(N)]
    ops[k+1] = op
    return tensor(ops)

# --- Hamiltonian: full homogeneous hyperfine ---
H = omega_e * e(sz)
for k in range(N):
    H += A * ( e(sp)*n(sm,k) + e(sm)*n(sp,k) )   # flip-flop terms
    H += A * e(sz) * n(sz,k)                    # Ising term (not essential)

# --- Initial state (electron up, one nucleus down enables flip-flop) ---
up, down = basis(2,0), basis(2,1)

nuc_state = tensor(down, up)      # |↓ ↑> for N=2
psi0 = tensor(up, nuc_state)      # electron ↑, one nucleus ↓

# --- Time evolution ---
tlist = np.linspace(0, 1, 200)

# Measure electron excited-state population
P_up_op = e(up*up.dag())

result = sesolve(H, psi0, tlist, e_ops=[P_up_op])

# --- Plot ---
import matplotlib.pyplot as plt
plt.plot(tlist, result.expect[0])
plt.xlabel("time")
plt.ylabel(r"$P_\uparrow(t)$")
plt.title("Electron–nuclear flip-flop")
plt.show()
